# Case 1: rainfall on DEM

With this notebook we will:
- Process the HydroDEM into H2Flow ASCI dem
- Define the settings for H2Flow grid generator
- Process ASCII rainfall to a H2Flow timeseries

In [1]:
import rasterio
from pathlib import Path
import geopandas as gpd
import pandas as pd
from rasterio.features import rasterize
import copy

# Prepare input data:

Prepare the input for this notebook by:
- downloading your data from: https://drive.google.com/drive/folders/1e3kjUWD-ikHo_TSmjCY86QyL-40R9xt3?usp=sharing.
- putting the data into ..\data\de_tol relative to the path-location of this notebook.

Below the input paths are defined, they should all exist

In [2]:
#input paths
data_path = Path(r"..\data\de_tol")
dem_tif = data_path.joinpath("dem_kockengen_20160905.tif")
wla_gpkg = data_path.joinpath("waterlevel_areas.gpkg")
gauges_gpkg = data_path.joinpath("gauges.gpkg")
rainfall_path = data_path.joinpath("rainfall")

# Define output and settings:

These will be used in writing output:
- dem_version: version number of the dem-file
- grid_version: version of the grid to be generated
- grid_size: grid_size for the computational grid in unit-sizes of the original dem cell-size

In [3]:
# project_directory
project_dir = data_path.joinpath("case")
project_dir.mkdir(parents=True, exist_ok=True)

# set all versions
dem_version = 1
grid_version = 1
finest_grid_size = 20

# Building the project-files:

Now we will start to do the actual work:
1. Read the GeoTIFF into memory
2. Convert the DEM into a H2F compatible ESRI ASCII file
3. Write the settings_file for the H2Flow grid_generator
4. Sample the Rainfall ESRI ASCII-files into a H2F compatible CSV-file

## 1. Read the GeoTIFF into memory

We read the original DEM (GeoTIFF) and change the profile that we'll use later to write the output to AAIGrid (ESRI ASCI grid).

In [4]:
with rasterio.open(dem_tif) as src:
    profile = src.profile
    data = src.read(1)

profile["driver"] = "AAIGrid"

print(profile)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 10274, 'height': 12146, 'count': 1, 'crs': CRS.from_epsg(28992), 'transform': Affine(0.5, 0.0, 124434.5,
       0.0, -0.5, 464846.0), 'tiled': False, 'compress': 'deflate', 'interleave': 'band'}


## 2. Convert the DEM into a H2F compatible ESRI ASCII file

We include the following parameters in the dem_file name and write it in the project_directory:
- profile["width"]: number of columns in the DEM
- profile["height"]: number of rows in the DEM
- dem_version: version of the DEM

In [5]:
dem_file = f'depthasci,imax= {profile["width"]},jmax= {profile["height"]},netw1d=  0.  {dem_version}'

with rasterio.open(project_dir.joinpath(dem_file), "w+", **profile) as dst:
    dst.write(data, 1)

## 3. Write the grid-generator settings-file

Below the settings that will be used by the grid-generator with the following information on each line:
- grid_version & dem_version: version of the DEM and the grid to be generated
- quadtree_levels: We will not generate a quadtree here, so keep it at 1
- finest_grid_size: The smallest grid-size in units of cell-size of the original DEM
- profile["height"] & profile["width"]
- sub-grid parameters: Defining accuracy on how the dem is processed in the H2Flow grid
- manning_friction: we will use a constant-value of 0.04 
- infiltration: we will use no infiltration here
- initial level: we will use a constant value of -1.8

In [6]:
quadtree_levels = 1
manning_value = 0.04
initial_level = -1.8

grid_gen_file = f"input_grid_gen.  {grid_version}"
grid_gen_path = project_dir.joinpath(grid_gen_file)

grid_gen_settings = f"""
  {grid_version}  {dem_version}  0   0 !grid number, depthfile, 1D network, Ridges number
  1  ! quadtree max. level
  20  ! finest coarse grid resolution (how many dem pixels do you lump together?)
 {profile["width"]}  ! ncols
 {profile["height"]}  ! nrows
 0.25  0.25  1.0    ! table_step_size,subgrid params,1d spacing
 0.04  1 ! manning value 
   0   1.0   20.0  !infiltration parameters - inactive with 0
  {initial_level}  0   ! initial water level default and file for s00, 0 means no files
  """

grid_gen_path.write_text(grid_gen_settings)

print(grid_gen_settings)


  1  1  0   0 !grid number, depthfile, 1D network, Ridges number
  1  ! quadtree max. level
  20  ! finest coarse grid resolution (how many dem pixels do you lump together?)
 10274  ! ncols
 12146  ! nrows
 0.25  0.25  1.0    ! table_step_size,subgrid params,1d spacing
 0.04  1 ! manning value 
   0   1.0   20.0  !infiltration parameters - inactive with 0
  -1.8  0   ! initial water level default and file for s00, 0 means no files
  


## 4. Sample the Rainfall ESRI ASCII-files into a H2F compatible CSV-file

Sample rainfall ASCII to series to a pandas dataframe and save it as H2Flow rainfall-CSV with:
- line 1: indicating a single rainfall gauge, and the level and time parameters to convert to rainfall rate (m/s)
- line 2: indicating the bounding box (xmin, ymin, xmax, ymax) of the rainfall gauge
- line 3+, col 0:4: the rainfall values: days, hours, minutes, seconds from start of simulation
- line 3+, col 5: the rainfall value in mm over the last timestep

In [7]:
m_conversion = 0.001
sec_conversion = -3600.0

rainfall_ascs = rainfall_path.glob("*.ASC")
gdf = gpd.read_file(gauges_gpkg)
xy = [(geom.x, geom.y) for geom in gdf["geometry"]]
columns = {"days": int, "hours": str, "minutes": str, "seconds": int, "value": float}
df = pd.DataFrame(columns=columns)

for idx, asc in enumerate(rainfall_ascs):
    datetime = pd.to_datetime(asc.stem, format="NSL_%Y%m%d_%H")

    if idx == 0:
        ref_datetime = copy.copy(datetime)

    time_delta = datetime - ref_datetime

    days = time_delta.components.days
    hours = f"{time_delta.components.hours:02d}"
    minutes = f"{time_delta.components.minutes:02d}"
    seconds = time_delta.components.seconds

    with rasterio.open(asc) as src:
        value = [val[0] for val in src.sample(xy)][0]
        df.loc[idx] = [days, hours, minutes, seconds, value]


rain_path = project_dir.joinpath("rain. -1")

header = f"""-1 {m_conversion} {sec_conversion}
 1  0.0  0.0  1.0e7  1.0e7
 """

rain_path.write_text(header)

df.astype(columns).to_csv(
    rain_path, mode="a", index=False, header=False, float_format="%.2f"
)

print(rain_path.read_text())

C:\Anaconda3\envs\work\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


-1 0.001 -3600.0
 1  0.0  0.0  1.0e7  1.0e7
 0,00,00,0,0.00
0,01,00,0,0.00
0,02,00,0,17.23
0,03,00,0,1.27
0,04,00,0,4.02
0,05,00,0,0.00
0,06,00,0,0.00
0,07,00,0,0.00
0,08,00,0,15.29
0,09,00,0,0.00
0,10,00,0,0.34
0,11,00,0,17.56
0,12,00,0,39.75
0,13,00,0,33.85
0,14,00,0,0.16
0,15,00,0,0.05
0,16,00,0,1.55
0,17,00,0,0.04
0,18,00,0,0.00
0,19,00,0,0.00
0,20,00,0,0.00
0,21,00,0,0.00
0,22,00,0,0.00
0,23,00,0,0.00
1,00,00,0,0.00



# This part will be deleted from this notebook and will be part of a new, more complex, case!!

### Water-level-areas to initial water-level ascii

We burn water-levels in the following order:
1. VASTPEIL (fixed level) as default
2. ZOMERPEIL (summer level) if 1 is nan
3. (ONDERPEIL + BOVENPEIL) / 2 if 2 is still nan

In [8]:
gdf = gpd.read_file(wla_gpkg, bbox=src.bounds)
gdf.loc[:, "LEVEL"] = gdf["VASTPEIL"]
gdf.loc[gdf["LEVEL"].isna(), "LEVEL"] = gdf["ZOMERPEIL"]
gdf.loc[gdf["LEVEL"].isna(), "LEVEL"] = (gdf["BOVENPEIL"] + gdf["ONDERPEIL"]) / 2

data = data * 0 + profile["nodata"]
shapes = ((row["geometry"], row["LEVEL"]) for _, row in gdf.iterrows())
rasterize(shapes, out=data, out_shape=src.res, transform=profile["transform"])

with rasterio.open(project_dir.joinpath("init_wl.asc"), "w+", **profile) as dst:
    dst.write(data, 1)

### Friction to ASCII

For now we apply a fixed manning roughness coefficient of 0.04 for the entire grid: https://www.engineeringtoolbox.com/mannings-roughness-d_799.html. Later we can vary with landuse/landcover.

In [9]:
data = data * 0  + 0.04

with rasterio.open(project_dir.joinpath("friction.asc"), "w+", **profile) as dst:
    dst.write(data, 1)